In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime, timedelta

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_MICH_BIB.csv'
df = pd.read_csv(filename, sep = ',', engine='python')

df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('#', 'num')
df.columns = df.columns.str.replace('%', 'per')

# df['RPB_MICH'] = df['RPB_MICH'].str.replace('$','')
# df['RPB_BFG_'] = df['RPB_BFG_'].str.replace('$','')
df['Business_Development_Fund'] = df['Business_Development_Fund'].str.replace(',','')
df['Business_Development_Fund'] = df['Business_Development_Fund'].str.replace(')','')
df['Business_Development_Fund'] = df['Business_Development_Fund'].str.replace('(','')
df['YTD_Flag_Brands'] = df['YTD_Flag_Brands'].str.replace(',','')
df['Business_Development_Fund'] = df['Business_Development_Fund'].str.replace('$','')
df['Loyalty_per'] = df['Loyalty_per'].str.replace('%','')

df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
#---------------------------------------------------------------------------------------------------------------------------#
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/MICH_AAD_BIB_BASIC_Incentive.txt'
df_tierMap_BASIC = pd.read_csv(filename2, sep = '\t')
df_tierMap_BASIC.columns = df_tierMap_BASIC.columns.str.replace(' ', '_')
#---------------------------------------------------------------------------------------------------------------------------#
filename3 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/MICH_AAD_BIB_PLUS_Incentive.txt'
df_tierMap_PLUS = pd.read_csv(filename3, sep = '\t')
df_tierMap_PLUS.columns = df_tierMap_PLUS.columns.str.replace(' ', '_')
# df.shape

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData) - timedelta(days=41) # USED FOR TESTING-POC *** THIS SHOULD BE CHANGED
tup_currentData = dt_currentData.timetuple()
# print(tup_currentData)

ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [ ]:
df['CreateDate'] = str(tup_currentData.tm_year) + '-' + str(tup_currentData.tm_mon) + '-' + str(tup_currentData.tm_mday)

In [ ]:
# df.columns.to_series(index=range(df.shape[1]))

In [ ]:
currentYear = datetime.now().year # use this for last transaction from G3X program
tup_now = datetime.now().timetuple() # used to create filename for CSV output

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    singleCusty_select = singleCusty[
        ['Dealer','BIB_Category','YTD_Flag_Brands','YTD_Michelin','YTD_BFG','YTD_UNI','Business_Development_Fund','CVP','Loyalty_Eligible']]
        
    if  singleCusty_select.Loyalty_Eligible.values == 'Yes':

        if (tup_currentData.tm_mon == 1) | (tup_currentData.tm_mon == 2) | (tup_currentData.tm_mon == 3):
            if ((tup_currentData.tm_year % 400 == 0) | (tup_currentData.tm_year % 4 == 0)) & (tup_currentData.tm_year % 100 != 0):
                daysINyear = 366
                print('Leap year')
            else:
                daysINyear = 365
        
        numDays = tup_currentData.tm_yday # get number of days
        currentQTY = singleCusty_select.YTD_Flag_Brands.astype(int).values
        currentRebate = (singleCusty_select.Business_Development_Fund.astype(float) + singleCusty_select.CVP).values
        numLocations = singleCusty.num_POS.astype(int).values
        
        estimatedQTY = np.floor(singleCusty_select.YTD_Flag_Brands.astype(int) / (numDays / daysINyear)).values
##########################################################CURRENT TIRES#####################################################
        # Check if customer is aligned
        if singleCusty_select.BIB_Category.values == 'BIB Basic':
            df_tierMap = df_tierMap_BASIC
            if   (currentQTY >= (df_tierMap.iloc[0,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[0,3] * numLocations)):
                currentTier = 0
                currentUP = 1
            else:
                currentTier = 1
                currentUP = 1

        else:
            df_tierMap = df_tierMap_PLUS
            if   (currentQTY >= (df_tierMap.iloc[0,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[0,3] * numLocations)):
                currentTier = 0
            elif (currentQTY >= (df_tierMap.iloc[1,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[1,3] * numLocations)):
                currentTier = 1
            elif (currentQTY >= (df_tierMap.iloc[2,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[2,3] * numLocations)):
                currentTier = 2
            elif (currentQTY >= (df_tierMap.iloc[3,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[3,3] * numLocations)):
                currentTier = 3
            elif (currentQTY >= (df_tierMap.iloc[4,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[4,3] * numLocations)):
                currentTier = 4
            elif (currentQTY >= (df_tierMap.iloc[5,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[5,3] * numLocations)):
                currentTier = 5
            elif (currentQTY >= (df_tierMap.iloc[6,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[6,3] * numLocations)):
                currentTier = 6
            elif (currentQTY >= (df_tierMap.iloc[7,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[7,3] * numLocations)):
                currentTier = 7
            elif (currentQTY >= (df_tierMap.iloc[8,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[8,3] * numLocations)):
                currentTier = 8
            elif (currentQTY >= (df_tierMap.iloc[9,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[9,3] * numLocations)):
                currentTier = 9
            elif (currentQTY >= (df_tierMap.iloc[10,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[10,3] * numLocations)):
                currentTier = 10
            elif (currentQTY >= (df_tierMap.iloc[11,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[11,3] * numLocations)):
                currentTier = 11
            elif (currentQTY >= (df_tierMap.iloc[12,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[12,3] * numLocations)):
                currentTier = 12
            elif (currentQTY >= (df_tierMap.iloc[13,2] * numLocations)) & (currentQTY <= (df_tierMap.iloc[13,3] * numLocations)):
                currentTier = 13
            else:
                currentTier = 14
            
            if currentTier < 14:
                currentUP = currentTier + 1
            else:
                currentUP = 14
#####################################################ESTIMATED TIRES#######################################################
        if singleCusty_select.BIB_Category.values == 'BIB Basic':
            df_tierMap = df_tierMap_BASIC
            if   (estimatedQTY >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            else:
                estTier = 1
            if estTier < 1:
                tierUP = estTier + 1
            else:
                tierUP = 1
        else:
            df_tierMap = df_tierMap_PLUS
            if   (estimatedQTY >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            elif (estimatedQTY >= (df_tierMap.iloc[1,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[1,3] * numLocations)):
                estTier = 1
            elif (estimatedQTY >= (df_tierMap.iloc[2,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[2,3] * numLocations)):
                estTier = 2
            elif (estimatedQTY >= (df_tierMap.iloc[3,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[3,3] * numLocations)):
                estTier = 3
            elif (estimatedQTY >= (df_tierMap.iloc[4,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[4,3] * numLocations)):
                estTier = 4
            elif (estimatedQTY >= (df_tierMap.iloc[5,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[5,3] * numLocations)):
                estTier = 5
            elif (estimatedQTY >= (df_tierMap.iloc[6,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[6,3] * numLocations)):
                estTier = 6
            elif (estimatedQTY >= (df_tierMap.iloc[7,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[7,3] * numLocations)):
                estTier = 7
            elif (estimatedQTY >= (df_tierMap.iloc[8,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[8,3] * numLocations)):
                estTier = 8
            elif (estimatedQTY >= (df_tierMap.iloc[9,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[9,3] * numLocations)):
                estTier = 9
            elif (estimatedQTY >= (df_tierMap.iloc[10,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[10,3] * numLocations)):
                estTier = 10
            elif (estimatedQTY >= (df_tierMap.iloc[11,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[11,3] * numLocations)):
                estTier = 11
            elif (estimatedQTY >= (df_tierMap.iloc[12,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[12,3] * numLocations)):
                estTier = 12
            elif (estimatedQTY >= (df_tierMap.iloc[13,2] * numLocations)) & (estimatedQTY <= (df_tierMap.iloc[13,3] * numLocations)):
                estTier = 13
            else:
                estTier = 14
            if estTier < 14:
                tierUP = estTier + 1
            else:
                tierUP = 14

############################################################################################################################
        aveTirePrice = (currentRebate / (df_tierMap.iloc[currentTier,1] / 100)) / currentQTY
        # floor to underestimate estimated tires
        bib_est_tires_rebate = estimatedQTY * df_tierMap.iloc[estTier,1]
        bib_1up_tires = df_tierMap.iloc[tierUP,2]
        bib_1up_tires_rebate = bib_1up_tires * df_tierMap.iloc[tierUP,1]
        units2nextTier = df_tierMap.iloc[currentUP,2] - currentQTY
        
        if bib_est_tires_rebate >= bib_1up_tires_rebate*numLocations:
            bib_1up_tires = estimatedQTY
            bib_1up_tires_rebate = bib_est_tires_rebate

        range_dollars_ll = ((bib_1up_tires*df_tierMap.iloc[0,1]))
        range_dollars_ul = ((bib_1up_tires*df_tierMap.iloc[tierUP,1]))

# ###########################################################################################################################        
#         print(singleCusty_select)
#         print('\nTier Map:\n',df_tierMap)
#         print('\nCurrent Tier:', currentTier)
#         print('\nCurrent QTY:', currentQTY)
#         print('\nCurrent Rebate:', currentRebate)
#         print('\nAverage Tire Price:', aveTirePrice)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estTier])        
#         print('\nEstimated Tires ~ bib:', estimatedQTY)        
#         print('\nEstimated Rebate ~ bib:',bib_est_tires_rebate)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])
#         print('\n1up Tires ~ bib:', bib_1up_tires*numLocations)
#         print('\n1up Dollars ~ bib:',bib_1up_tires_rebate*numLocations)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul*numLocations, '\n')
# ###########################################################################################################################
#         print(tup_currentData, '\n' 'indexNumber:', ii, '\n')
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################

        tempData = pd.DataFrame(
            {'Est_tier':estTier, 'Current_tier': currentTier, 'Current_QTY': currentQTY, 'Current_rebate': currentRebate,
             'Ave_tire_price': aveTirePrice, 'Units2nextTier':units2nextTier,
             'estimated_tires':estimatedQTY, 'estimated_rebate':bib_est_tires_rebate,
             'Up_tier':tierUP,
             'bib_up_tires':bib_1up_tires*numLocations, 'bib_up_rebate':bib_1up_tires_rebate*numLocations,
             'Range_lower':range_dollars_ll, 'Range_upper':range_dollars_ul*numLocations,'Grain': 'Yearly'})
        calculatedData = calculatedData.append(tempData)
    else:
        tempData = pd.DataFrame(
            {'Est_tier':estTier, 'Current_tier':currentTier, 'Current_QTY':currentQTY, 'Current_rebate':'Below 75%',
             'Ave_tire_price':'Below 75%', 'Units2nextTier':units2nextTier,
             'estimated_tires':estimatedQTY, 'estimated_rebate':'Below 75%',
             'Up_tier':tierUP,
             'bib_up_tires':bib_1up_tires*numLocations, 'bib_up_rebate':'Below 75%',
             'Range_lower':range_dollars_ll, 'Range_upper':range_dollars_ul,'Grain': 'Yearly'})
        calculatedData = calculatedData.append(tempData)
#     print(ii)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.drop(finalData[finalData.Dealer.values == 0].index)
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(172.12)
# finalData.shape

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_MICH_BIB.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_MICH_BIB_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)